In [ ]:
# 1. Imports & Config
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from src.data_loader       import load_all_tickers
from src.slippage_simulator import build_impact_curve
from src.model_fit        import fit_power_law, fit_alpha_time_varying
from src.execution_schedule import static_schedule, rolling_schedule

# 2. Load Data
paths = {
    'TICKA': 'data/TICKA.csv',
    'TICKB': 'data/TICKB.csv',
    'TICKC': 'data/TICKC.csv'
}
lob = load_all_tickers(paths)

# 3. Build Impact Curves
size_grid = np.arange(10, 360, 10)
impact_curves = {}
for tic,df in lob.items():
    x,slip = build_impact_curve(df, size_grid, side='buy')
    impact_curves[tic] = (x, slip)
    plt.loglog(x, slip, label=tic)
plt.xlabel('Order size')
plt.ylabel('Avg slippage')
plt.legend(); plt.show()

# 4. Fit Power-Law
fits = {}
for tic,(x,slip) in impact_curves.items():
    α,γ,R2 = fit_power_law(x, slip)
    fits[tic] = (α,γ,R2)
    print(f"{tic}: α={α:.4f}, γ={γ:.2f}, R²={R2:.2f}")

# 5. Time-Varying αₜ
# Example for one ticker
df = lob['TICKA']
# calculate spread and depth
df['spread'] = df['ask_prices'].apply(lambda p: p[0]) - df['bid_prices'].apply(lambda p: p[0])
df['depth']  = df['bid_sizes'].apply(lambda s: sum(s)) + df['ask_sizes'].apply(lambda s: sum(s))
# estimate αₜ from impact curves (invert x^γ)
α_t_est = []
for _,row in df.iterrows():
    # pick small x to proxy αₜ: αₜ ≈ gₜ(x)/x^γ
    α_t_est.append(simulate_slippage(row, size_grid[0],'buy')/size_grid[0]**fits['TICKA'][1])
α_t_est = np.array(α_t_est)
β, R2_tv = fit_alpha_time_varying(df['spread'].values, df['depth'].values, α_t_est)
print("Time-varying α params:", β, "R²:", R2_tv)

# 6. Build Schedule
α_forecast = β[0] + β[1]*df['spread'] + β[2]/df['depth']
S = 1000
x_static  = static_schedule(α_forecast.values, fits['TICKA'][1], S)
x_rolling = rolling_schedule(α_forecast.values, fits['TICKA'][1], S)

plt.plot(x_static, label='static')
plt.plot(x_rolling, label='rolling')
plt.legend(); plt.show()
